# Unit 5 - Financial Planning


In [ ]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [ ]:
# Load .env enviroment variables
load_dotenv()

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets
my_btc = 1.2
my_eth = 5.3

In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

In [ ]:
# Fetch current BTC price
res = requests.get(btc_url)
btc_value = res.json()["data"]["1"]["quotes"]["USD"]["price"]

# Fetch current ETH price
res = requests.get(eth_url)
eth_value = res.json()["data"]["1027"]["quotes"]["USD"]["price"]
# Compute current value of my crpto
my_btc_value = btc_value * my_btc
my_eth_value = eth_value * my_eth

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Current amount of shares
my_agg = 200
my_spy = 50

In [ ]:
# Set Alpaca API key and secret
APCA_API_BASE_URL = "https://paper-api.alpaca.markets"
APCA_API_KEY_ID = "PKEFE0RIZWTBJYZO3586"
APCA_API_SECRET_KEY = "9tCDvzED35KgQFsz5aV8uwzzyHQslYLIrvkIOqsx"
# Create the Alpaca API object
api = tradeapi.REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY)

In [ ]:
# Format current date as ISO format
import datetime
current_date = datetime.datetime.now().isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
df = api.get_barset(tickers, timeframe, limit=1, start=current_date).df

# Preview DataFrame
print(df['AGG'].close)

In [ ]:
# Pick AGG and SPY close prices
agg_close_price = df['AGG']['close'][0]
spy_close_price = df['SPY']['close'][0]
# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")

In [ ]:
# Compute the current value of shares
my_spy_value = spy_close_price * my_spy
my_agg_value = agg_close_price * my_agg

# Print current value of share
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income
monthly_income = 12000

# Create savings DataFrame
data = [ my_btc_value + my_eth_value, my_agg_value + my_spy_value]
df_savings = pd.DataFrame(data, columns = ["amount"], index =["crypto", "shares"])

# Display savings DataFrame
display(df_savings)

In [ ]:
# Plot savings pie chart
plot = df_savings.plot.pie(y="amount", figsize=(5, 5))

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
savings = df_savings['amount'].sum()

# Validate saving health
if savings > emergency_fund:
    print("Congratulations!! You have enough funds")
elif savings == emergency_fund:
    print("Congratulations!! You have reached you goal")
else:
    print(emergency_fund - savings, " is the amount that remains to reach you goal")

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2015-08-07', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2020-08-07', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
df_stock_data = api.get_barset(tickers, timeframe, start=start_date, end=end_date).df

# Display sample data
df_stock_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
MC_30 = MCSimulation(portfolio_data = df_stock_data, weights = [.40,.60], num_simulation = 500, num_trading_days = 252*30)

In [ ]:
# Printing the simulation input data
MC_30.portfolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
MC_30.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
MC_30.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
MC_30.plot_distribution()

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
summary = MC_30.summarize_cumulative_return()
# Print summary statistics
print(summary)

### Calculate the expected portfolio return at the 95% lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower = round(summary[8]*20000,2)
ci_upper = round(summary[9]*20000,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
ci_lower = round(summary[8]*30000,2)
ci_upper = round(summary[9]*30000,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5 = MCSimulation(portfolio_data = df_stock_data, weights = [.40,.60], num_simulation = 500,num_trading_days = 252*5)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
MC_5.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
MC_5.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
summary = MC_5.summarize_cumulative_return()

# Print summary statistics
print(summary)

In [ ]:
# Set initial investment
initial_investment = 20000 * 3

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_five = round(summary[8]*60000,2)
ci_upper_five = round(summary[9]*60000,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
MC_10 = MCSimulation(portfolio_data = df_stock_data, weights = [.40,.60], num_simulation = 500,num_trading_days = 252*5)

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
MC_10.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
MC_10.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
MC_10.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
summary = MC_10.summarize_cumulative_return()

# Print summary statistics
print(summary)

In [ ]:
# Set initial investment
initial_investment = 20000 * 3

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_ten = round(summary[8]*60000,2)
ci_upper_ten = round(summary[9]*60000,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")